<a href="https://colab.research.google.com/github/Flychuban/9grade_C/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00


In [2]:
import requests
from transformers import pipeline
from bs4 import BeautifulSoup
import re

In [3]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [4]:
def search_urls_space_news():
  URL = "https://www.google.com/search?q=nasa+news&tbm=nws"
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [5]:
hrefs = search_urls_space_news()

In [6]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [7]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [28]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.nasa.gov/press-release/nasa-s-first-asteroid-sample-has-landed-now-secure-in-clean-room',
 'https://www.aljazeera.com/gallery/2023/9/25/major-asteroid-sample-brought-to-earth-in-nasa-first',
 'https://www.npr.org/2023/09/24/1201386042/watch-live-nasa-sends-an-asteroid-sample-back-to-earth',
 'https://www.foxweather.com/earth-space/nasa-osiris-rex-asteroid-sample-earth-landing-utah',
 'https://www.space.com/nasa-osiris-rex-dante-lauretta-parachute-emotions',
 'https://abcnews.go.com/US/video/nasa-celebrating-end-7-year-mission-103464365',
 'https://abcnews.go.com/US/nasas-osiris-rex-asteroid-sample-return-mission/story%3Fid%3D103408191',
 'https://www.google.com/search?q%3Dnasa%2Bnews%26tbm%3Dnws%26pccc%3D1',
 'https://www.cnn.com/videos/business/2023/09/25/nasa-asteroid-sample-osiris-rex-cprog-cnntm-vpx.cnn',
 'https://www.nbcnews.com/science/space/first-nasa-returns-asteroid-samples-earth-rcna111474',
 'https://www.cnn.com/2023/09/24/world/osiris-rex-asteroid-sample-retu

In [9]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [39]:
titles, articles = get_text_from_urls(cleaned_urls)

In [40]:
articles

['In Pictures A seven-year space voyage came to its climactic end\xa0when a NASA capsule landed in the desert in the US state of Utah, carrying to Earth the largest asteroid sample ever collected. Scientists have high hopes for the sample, which landed on Sunday, saying it will provide a better understanding of the formation of our solar system and how Earth became habitable. The 6.21-billion-km (3.86-billion-mile) journey marked the United States’ first sample return mission of its kind, the US space agency said in a post on X, formerly known as Twitter. NASA chief Bill Nelson hailed the mission and said the asteroid dust “will give scientists an extraordinary glimpse into the beginnings of our solar system”. The OSIRIS-REx probe sent its sample capsule into a fiery, perilous descent through Earth’s atmosphere, but NASA managed to engineer a soft landing at 8:52am (14:52 GMT) in the military’s Utah Test and Training Range. Four years after its 2016 launch, the probe had landed on the 

Split text from the article into chunks

In [41]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))
sentences[-1]

['Advertisement Supported by The seven-year Osiris-Rex mission ended on Sunday with the return of regolith from the asteroid Bennu, which might hold clues about the origins of our solar system and life.',
 ' transcript “We have confirmed parachute deployment.',
 '” “Wow, and after an exhilarating streak across Earth’s atmosphere, we have parachute deployment.',
 ' You can see just a sigh of relief from the team.',
 ' I can hear some applause here.',
 ' You see the reaction there just moments ago as they got that sample back on the ground.',
 ' The rest of the team members approaching that S.',
 'R.',
 'C.',
 ' that has been successfully bagged and prepared to be loaded into the helicopter’s long line.',
 ' And we have clearance to depart the recovery site.',
 ' This is a key moment of those recovery operations.',
 ' We’re already getting that S.',
 'R.',
 'C out of the landing zone and on its way to the Dugway clean room.',
 ' In just a few moments, the SR.',
 'C.',
 ' will be back on 

In [42]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[i][current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0


In [43]:
chunks[0][0]

['In',
 'Pictures',
 'A',
 'seven-year',
 'space',
 'voyage',
 'came',
 'to',
 'its',
 'climactic',
 'end\xa0when',
 'a',
 'NASA',
 'capsule',
 'landed',
 'in',
 'the',
 'desert',
 'in',
 'the',
 'US',
 'state',
 'of',
 'Utah,',
 'carrying',
 'to',
 'Earth',
 'the',
 'largest',
 'asteroid',
 'sample',
 'ever',
 'collected.',
 '',
 'Scientists',
 'have',
 'high',
 'hopes',
 'for',
 'the',
 'sample,',
 'which',
 'landed',
 'on',
 'Sunday,',
 'saying',
 'it',
 'will',
 'provide',
 'a',
 'better',
 'understanding',
 'of',
 'the',
 'formation',
 'of',
 'our',
 'solar',
 'system',
 'and',
 'how',
 'Earth',
 'became',
 'habitable.',
 '',
 'The',
 '6.',
 '21-billion-km',
 '(3.',
 '86-billion-mile)',
 'journey',
 'marked',
 'the',
 'United',
 'States’',
 'first',
 'sample',
 'return',
 'mission',
 'of',
 'its',
 'kind,',
 'the',
 'US',
 'space',
 'agency',
 'said',
 'in',
 'a',
 'post',
 'on',
 'X,',
 'formerly',
 'known',
 'as',
 'Twitter.',
 '',
 'NASA',
 'chief',
 'Bill',
 'Nelson',
 'hailed

In [44]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [57]:
joined_chunks[6]

['\nEditor’s Note: Sign up for CNN’s Wonder Theory science newsletter. \xa0Explore the universe with news on fascinating discoveries, scientific advancements and more. \n \n      An asteroid sample collected by NASA has touched down on Earth, giving scientists the opportunity to learn more about the origins of the solar system and capturing a piece of a massive space rock that has a chance of colliding with our planet in the future.  It’s the first time the agency has accomplished such a feat. \n   \n      Seven years after launching to space, the OSIRIS-REx spacecraft flew by Earth Sunday to deliver the pristine sample from the near-Earth asteroid Bennu. \n   \n      OSIRIS-REx, which stands for Origins, Spectral Interpretation, Resource Identification, Security, Regolith Explorer, lifted off in 2016 and began orbiting Bennu in 2018.  The spacecraft collected the sample in 2020 and set off on its lengthy return trip to Earth in May 2021.  The mission traveled 3. 86 billion miles total